In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/handwritten-digit-identification-b1/sample_submission.csv
/kaggle/input/handwritten-digit-identification-b1/train.csv
/kaggle/input/handwritten-digit-identification-b1/test.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/handwritten-digit-identification-b1/train.csv',header=None)
test_data=pd.read_csv('/kaggle/input/handwritten-digit-identification-b1/test.csv',header=None)

In [3]:
data =train_data
features = data.drop(columns=[65])
labels = data[65]

In [4]:
correlation = data.corr()
target_correlation = correlation[65].abs().sort_values(ascending=False)
top_features = target_correlation[1:55].index  
selected_data = data[top_features]

In [5]:
# Correlation Heatmap
plt.figure(figsize=(100, 100))
correlation = features.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Pixel Values')
plt.show()

In [6]:
print("Selected Features:")
print(top_features)

Selected Features:
Int64Index([53, 36, 28, 13, 29, 34, 15,  6,  7, 61, 30, 54, 64, 52, 26, 37, 20,
            21, 38, 51, 42, 63, 12,  5, 22, 11, 43,  8, 56, 62, 19, 16, 14, 23,
            27, 47, 48, 58,  9,  2, 50, 25, 24, 18, 57, 17, 31, 60, 45,  4, 49,
             3, 41, 59],
           dtype='int64')


In [7]:
train_df=train_data[[53, 36, 28, 13, 29, 34, 15,  6,  7, 61, 30, 54, 64, 52, 26, 37, 20,
            21, 38, 51, 42, 63, 12,  5, 22, 11, 43,  8, 56, 62, 19, 16, 14, 23,
            27, 47, 48, 58,  9,  2, 50, 25, 24, 18, 57, 17, 31, 60, 45,  4, 49,
             3, 41, 59]]
test_df=test_data[[53, 36, 28, 13, 29, 34, 15,  6,  7, 61, 30, 54, 64, 52, 26, 37, 20,
            21, 38, 51, 42, 63, 12,  5, 22, 11, 43,  8, 56, 62, 19, 16, 14, 23,
            27, 47, 48, 58,  9,  2, 50, 25, 24, 18, 57, 17, 31, 60, 45,  4, 49,
             3, 41, 59]]

In [8]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,1431,0,0,0,2,14,5,0,0,0,...,0,0,0,0,0,3,15,6,0,0
1,1432,0,0,0,13,12,0,0,0,0,...,12,0,0,0,1,10,16,14,4,0
2,1433,0,0,7,16,16,16,6,0,0,...,0,0,0,0,7,15,1,0,0,0
3,1434,0,2,15,15,6,0,0,0,0,...,1,0,0,3,15,14,11,2,0,0
4,1435,0,0,0,10,9,0,0,0,0,...,0,0,0,0,0,13,10,0,0,0


In [9]:
train_df.head()

,53,36,28,13,29,34,15,6,7,61,...,57,17,31,60,45,4,49,3,41,59
0,1,6,15,5,10,0,0,15,0,16,...,0,0,0,15,2,11,0,3,0,3
1,4,12,16,15,16,1,0,3,0,14,...,0,0,1,10,4,10,0,0,0,2
2,12,14,9,8,11,4,0,8,0,8,...,0,0,3,10,0,10,0,3,0,2
3,15,1,0,8,11,0,15,16,8,11,...,0,0,1,16,14,11,0,5,0,7
4,9,1,0,4,3,0,12,16,4,13,...,0,0,6,12,8,12,0,4,0,3


In [10]:
test_df.head()

,53,36,28,13,29,34,15,6,7,61,...,57,17,31,60,45,4,49,3,41,59
0,13,14,3,11,4,14,0,5,0,15,...,0,0,2,3,8,2,0,0,0,0
1,13,15,10,4,0,8,0,0,0,16,...,0,0,0,10,13,13,0,0,0,1
2,12,5,16,5,13,11,0,16,6,1,...,0,0,0,15,16,16,0,7,0,7
3,13,0,3,16,16,0,0,0,0,11,...,0,0,0,14,7,15,0,15,0,15
4,15,11,7,13,0,0,0,0,0,10,...,0,0,0,13,16,10,0,0,0,0


In [11]:
X_train=train_df
y_train=train_data[65]

In [12]:
#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train)
#test_df = scaler.transform(test_df)

In [13]:
# Define MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(800,), activation='logistic', solver='adam', 
                    learning_rate='adaptive', max_iter=200, random_state=42)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(mlp, X_train, y_train, cv=kfold)
scores

array([0.96853147, 0.97202797, 0.96153846, 0.99300699, 0.98251748])

In [14]:
mlp.fit(X_train,y_train)
y_pred=mlp.predict(test_df)

In [15]:
submission = pd.DataFrame({"ID": test_data[0], "Category": y_pred})
submission.to_csv("submission.csv", index=False)